# Orchestrazione completa del flusso di Ticket Triage Automatico

Questo notebook ha lo scopo di orchestrare end-to-end l’intero prototipo di **classificazione automatica dei ticket di assistenza**, sviluppato nell’ambito del Project Work.

In un unico flusso riproducibile vengono eseguite tutte le fasi principali:
- analisi di un dataset sintetico di ticket e suo split in sotto dataset di training e test,
- addestramento dei modelli di Machine Learning prescelti e valutazione delle prestazioni,
- comparazione dei modelli e salvataggio degli artefatti.

Il notebook non introduce nuova logica applicativa, ma coordina moduli già definiti ed eseguibili singolarmente, fungendo da punto di ingresso unico per l’esecuzione completa del progetto.


## Inizializzazione dell’ambiente di lavoro

In questa sezione vengono importate le librerie necessarie e definite le variabili di contesto condivise (percorsi di progetto, directory dati e output).

L’obiettivo è garantire:
- indipendenza dal sistema operativo,
- coerenza dei path tra notebook e script,
- possibilità di esecuzione del flusso da un unico punto.


In [145]:
import sys
print(sys.executable)

from pathlib import Path
from datetime import datetime
import papermill as pm

c:\project-work\.venv\Scripts\python.exe


In [146]:
PROJECT_ROOT = Path("..").resolve()  # perché sei dentro /notebooks
NOTEBOOKS_DIR = PROJECT_ROOT / "notebooks"
RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")

OUTPUT_DIR = PROJECT_ROOT / "runs" / RUN_ID
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("RUN_ID:", RUN_ID)
print("OUTPUT_DIR:", OUTPUT_DIR)


PROJECT_ROOT: C:\project-work
RUN_ID: 20260105_230145
OUTPUT_DIR: C:\project-work\runs\20260105_230145


# Predisposizione della pipeline 

Tutti i notebooks vengono inseriti in pipeline e vengono predisposte le condizione di esecuzione.


In [147]:
pipeline = [
    "01_exploration_dataset.ipynb",
    "02_preprocessing_dataset.ipynb",
    "03_TFIDF_Logic_Regression.ipynb",
    "04_TFIDF_LinearSVM.ipynb",
    "05_Naive_Bayes_Exploration.ipynb",
    "06_Model_Result_Compare.ipynb",
]

for nb in pipeline:
    nb_path = NOTEBOOKS_DIR / nb
    if not nb_path.exists():
        raise FileNotFoundError(f"Notebook non trovato: {nb_path}")

print("Notebook in pipeline:", len(pipeline))


Notebook in pipeline: 6


In [148]:
def run_notebook(input_name: str) -> Path:
    input_path = NOTEBOOKS_DIR / input_name
    output_path = OUTPUT_DIR / input_name.replace(".ipynb", "_out.ipynb")

    print(f"\n=== ESEGUO: {input_name} ===")
    pm.execute_notebook(
        input_path=str(input_path),
        output_path=str(output_path),
        cwd=str(PROJECT_ROOT),  
    )
    print(f"OK: salvato -> {output_path}")
    return output_path


## Esecuzione sequenziale della pipeline di notebook

In questa sezione viene eseguita l’intera pipeline del progetto attraverso il lancio sequenziale dei notebook che compongono il flusso di lavoro.

Ogni notebook rappresenta una fase autonoma e logicamente coerente del processo (ad esempio split dei dati, addestramento dei modelli, valutazione).  
L’esecuzione avviene in modo controllato e ordinato, garantendo che ogni step venga completato prima di passare al successivo.

I notebook eseguiti correttamente vengono tracciati e riportati a fine esecuzione, fornendo un riscontro immediato sul completamento dell’intera pipeline.


In [149]:
executed = []

for nb in pipeline:
    out = run_notebook(nb)
    executed.append(out)

print("\nPipeline completata. Notebook eseguiti:")
for p in executed:
    print("-", p)



=== ESEGUO: 01_exploration_dataset.ipynb ===


Executing: 100%|██████████| 29/29 [00:09<00:00,  2.98cell/s]


OK: salvato -> C:\project-work\runs\20260105_230145\01_exploration_dataset_out.ipynb

=== ESEGUO: 02_preprocessing_dataset.ipynb ===


Executing: 100%|██████████| 16/16 [00:05<00:00,  2.73cell/s]


OK: salvato -> C:\project-work\runs\20260105_230145\02_preprocessing_dataset_out.ipynb

=== ESEGUO: 03_TFIDF_Logic_Regression.ipynb ===


...

OK: salvato -> C:\project-work\runs\20260105_230145\04_TFIDF_LinearSVM_out.ipynb

=== ESEGUO: 05_Naive_Bayes_Exploration.ipynb ===


Executing: 100%|██████████| 28/28 [00:07<00:00,  3.58cell/s]


OK: salvato -> C:\project-work\runs\20260105_230145\05_Naive_Bayes_Exploration_out.ipynb

=== ESEGUO: 06_Model_Result_Compare.ipynb ===


Executing: 100%|██████████| 3/3 [00:07<00:00,  2.42s/cell]

OK: salvato -> C:\project-work\runs\20260105_230145\06_Model_Result_Compare_out.ipynb

Pipeline completata. Notebook eseguiti:
- C:\project-work\runs\20260105_230145\01_exploration_dataset_out.ipynb
- C:\project-work\runs\20260105_230145\02_preprocessing_dataset_out.ipynb
- C:\project-work\runs\20260105_230145\03_TFIDF_Logic_Regression_out.ipynb
- C:\project-work\runs\20260105_230145\04_TFIDF_LinearSVM_out.ipynb
- C:\project-work\runs\20260105_230145\05_Naive_Bayes_Exploration_out.ipynb
- C:\project-work\runs\20260105_230145\06_Model_Result_Compare_out.ipynb


## Tracciamento dell’esecuzione della pipeline

Al termine dell’esecuzione della pipeline, viene generato un file di log che documenta in modo persistente l’esito del run corrente.

Il log contiene:
- un identificativo univoco di esecuzione (RUN_ID),
- l’elenco dei notebook eseguiti correttamente.

All'interno della cartella di archivio run, identificata tramite la codice di data e ore di esecuzione (aaaammgg_hhmmss):
- file di log di esecuzione,
- file di dataset grezzo utilizzato,
- output dei singoli notebooks.

I dati storicizzati permetteranno una analisi a posteriori, confronti ed eventuali rielaborazioni.


In [150]:
log_path = OUTPUT_DIR / "run_log.txt"

with open(log_path, "w", encoding="utf-8") as f:
    f.write(f"RUN_ID: {RUN_ID}\n")
    f.write("Executed notebooks:\n")
    for p in executed:
        f.write(f"- {p.name}\n")

print("Log salvato in:", log_path)


Log salvato in: C:\project-work\runs\20260105_230145\run_log.txt
